In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
yrs=[1983,1984,1985]
arr=[]
links=[]
for yr in yrs:
    html=requests.get("https://www.profootballarchives.com/"+str(yr)+"usfl.html")
    soup=BeautifulSoup(html.text)
    stats=soup.find("div",{"id":"stats"})
    table=stats.find_all("table")[0]
    for a in table.find_all("a"):
        links.append(a.get("href"))
        url="https://www.profootballarchives.com/"+str(a.get("href"))
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[4]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(yr)
            arr.append(h)
yrs=[2022]
links=[]
html=requests.get("https://www.profootballarchives.com/2022usflbir.html")
soup=BeautifulSoup(html.text)
table=soup.find_all("table")[3]
for a in table.find_all("a"):
    if '<a></a>' not in str(a):
        links.append(a.get("href"))
for link in links:
    for yr in yrs:
        
        link=str(link)
        if '83' in link:
            link=link.split("83")
            link=link[1]
        if '84' in link:
            link=link.split("84")
            link=link[1]
        if '85' in link:
            link=link.split("85")
            link=link[1]
        link=link
        url="https://www.profootballarchives.com/"+link
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[3]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(2022)
            arr.append(h)
df=pd.concat([pd.DataFrame(arr)])
df.columns=['Day','Date','VenueType','team2','score','result','OT','Location','Venue','Attendance','team1','season']
df=df.astype({'VenueType':'string'})

df=df[df.Date.notna()]
df=df[df.score.notna()]
df['score2']=0
df['score1']=0
for row in df.itertuples():
    score=str(row.score)
    score=score.split("-")
    s1=score[0]
    s2=score[1]
    df.at[row.Index,'score1']=s1
    df.at[row.Index,'score2']=s2
df['Date']=df['Date'].str.replace("/","")
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,"Date":"string"})  
df['team1']=df['team1'].str.replace(" ","")
df['team2']=df['team2'].str.replace(" ","")
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len(f)>0:
        df=df.drop(row.Index)


In [1]:
f

NameError: name 'f' is not defined

In [47]:
import math
import numpy as np
import pandas as pd
df=pd.read_csv("https://github.com/easella/usfl/raw/main/usflscores.csv")
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,'Date':"string"})
df['team1']=df['team1'].str.replace(" ","")
df['team2']=df['team2'].str.replace(" ","")
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len(f)>0:
        df=df.drop(row.Index)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
hfa=[]
w=0

df.sort_values(by='Date',inplace=True)
l=0
gh={}
gh['g']=0
class Elo:
  global winnerHome
  def __init__(self,k,g=1,homefield = 182.5):
    self.ratingDict   = {}  
    self.k        = k
    self.g        = g
  def addPlayer(self,name,rating = 1500):
    self.ratingDict[name] = rating
  def gameOver(self, winner, loser, winnerHome,neutral,wins,loses):
    homef=gh['g']
    wb=0
    lb=0
    if winnerHome and neutral!='N':
      result = self.expectResult(self.ratingDict[winner] + homef, self.ratingDict[loser])
    if winnerHome==False and neutral!='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser] + homef)
    if neutral=='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser])
    wins=float(wins)
    loses=float(loses)
    if wins==loses:
        tie=True
    else:
        tie=False
    if tie:
        mult=(math.log(0+1))*(2.2/1*0.001+2.2)
        shift=(self.k*mult)*(0.5 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
    if tie !=True:
        win=eloLeague.ratingDict[winner]
        lose=eloLeague.ratingDict[loser]
        if neutral==True:
          win=(eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome and neutral!=True:
          win=(homef+eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome!=True and neutral!=True:
          lose=(homef+eloLeague.ratingDict[loser])
          win=(eloLeague.ratingDict[winner])
        global zy
        if(win>lose):
          zy=(win-lose)
        if(win==lose):
          zy=(win-lose)
        if(win<lose):
          zy=(win-lose)
        mult=(math.log((wins-loses)+1))*(2.2/(zy)*0.001+2.2)
        shift=(self.k*mult)*(1 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
  def expectResult(self, p1, p2):
        global exp
        exp = (p2-p1)/400
        n2=10
        j=exp
        o=1/((n2**(j))+1)
        return float(o)
teams=set(df.team1.tolist()+df.team2.tolist())

eloLeague=Elo(k=25)
currSeason=1983
q=pd.read_html("https://en.wikipedia.org/wiki/2023_USFL_season")[2]
q.columns = q.columns.droplevel(1)
q=q.replace("[a]","")
q=q.rename(columns={'Team[10]':'Team'})
q=q.rename(columns={'Home stadium':'Venue'})
venues={}
venues[2023]={}
for row in q.itertuples():
    l=list(row)
    team=str(l[1])
    team=team.replace("[a]","")
    team=team.replace(" ","")
    team=team.replace("MemphisShowboats","TampaBayBandits")
    venues[2023][team]=row.Venue


    
q=pd.read_html("https://en.wikipedia.org/wiki/United_States_Football_League_(2022)#Teams")[2]
q.columns = q.columns.droplevel(1)
q=q.replace("[a]","")
q=q.rename(columns={'Team[72]':'Team'})
q=q.rename(columns={'Stadium':'Venue'})
venues[2022]={}
for row in q.itertuples():
    l=list(row)
    team=str(l[1])
    team=team.replace("[a]","")
    team=team.replace(" ","")
    team=team.replace("MemphisShowboats","TampaBayBandits")

    venues[2022][team]=row.Venue
t=pd.read_csv("https://ontheroadtovote.com/ncaab/usflscores.txt")
df=pd.concat([df,t])
for row in df.itertuples():
    if row.season>2021:
        if venues[row.season][row.team2]==venues[row.season][row.team1]:
            df.at[row.Index,'VenueType']='N'
df=df.drop(['Unnamed: 0'], axis=1)
df=df.drop_duplicates()
hfa=[]
df['score2']=0
df['score1']=0

for row in df.itertuples():
    score=str(row.score)
    score=score.split("-")
    s1=score[0]
    s2=score[1]
    if row.season==2023:
        df.at[row.Index,'score1']=s2
        df.at[row.Index,'score2']=s1
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,'Date':"string",'VenueType':'string'})
n=df[df.VenueType!='N']
hfaz=[]
for row in n.itertuples():
    if row.VenueType=='A':
        hfaz.append(row.score1-row.score2)
    else:
        hfaz.append(row.score2-row.score1)
homef=round(np.mean(hfaz))
for team in teams:
    eloLeague.addPlayer(team,rating=1500)
gh['g']=round(np.mean(hfaz)*25)
l=0
w=0
df=df.replace("MemphisShowboats","TampaBayBandits")
for row in df.itertuples():
    if float(row.season)>currSeason:
        season=float(row.season)
        
                
       
            
       
        
        
        for key in eloLeague.ratingDict.keys():
            
            
                
            pre=eloLeague.ratingDict[key]
            if eloLeague.ratingDict[key]!=1500:
                eloLeague.ratingDict[key]=(eloLeague.ratingDict[key]*(2/3))+(1505*(1/3))
            af=eloLeague.ratingDict[key]
    
                
        currSeason+=1
    if row.score1>row.score2 or row.score1==row.score2:
        hfa=0
        if row.VenueType!='N':
            hfa=float(gh['g'])
        if row.VenueType=='A':

            if (eloLeague.ratingDict[row.team1]+hfa)>(eloLeague.ratingDict[row.team2]):
                w+=1
            if (eloLeague.ratingDict[row.team1]+hfa)<(eloLeague.ratingDict[row.team2]):
                l+=1
            winnerhome=True
            
        if row.VenueType==('H' or 'N'):
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]+hfa):
                l+=1
            winnerhome=False
        eloLeague.gameOver(row.team1,row.team2,winnerhome,row.VenueType,row.score1,row.score2)
    if row.score1<row.score2:
        hfa=0
        if row.VenueType!='N':
            hfa=gh['g']
        if row.VenueType=='A':
            if (eloLeague.ratingDict[row.team1]+hfa)<(eloLeague.ratingDict[row.team2]):
                w+=1
            if (eloLeague.ratingDict[row.team1]+hfa)>(eloLeague.ratingDict[row.team2]):
                l+=1
            winnerhome=False
        if row.VenueType=='H':
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                l+=1
            winnerhome=True
        if row.VenueType=='N':
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                l+=1
            winnerhome=True
        eloLeague.gameOver(row.team2,row.team1,winnerhome,row.VenueType,row.score2,row.score1)
css=df[df.season==2023]
if len(css)==0:
    lsdf=df[df.season<2023]
    ls=set(lsdf.team1.tolist()+lsdf.team2.tolist())
    csdf=df[df.season<2023]
    cs=set(csdf.team1.tolist()+csdf.team2.tolist()) 
    for key in eloLeague.ratingDict.keys():
            
        if(key  in ls and key in cs):
            pre=eloLeague.ratingDict[key]
            eloLeague.ratingDict[key]=(eloLeague.ratingDict[key]*(2/3))+(1500*(1/3))
            af=eloLeague.ratingDict[key]
            print(pre,af,key)
    

In [45]:
np.mean(hfaz)

0.08928571428571429

In [43]:
pd.read_html("https://en.wikipedia.org/wiki/2023_USFL_season")[2]

,Team[11],Head coach,Hub city,Home stadium
,North Division,North Division,North Division,North Division
0,Michigan Panthers,Mike Nolan,"Detroit, Michigan",Ford Field
1,Philadelphia Stars[a],Bart Andrus,"Detroit, Michigan",Ford Field
2,New Jersey Generals,Mike Riley,"Canton, Ohio",Tom Benson Hall of Fame Stadium
3,Pittsburgh Maulers,Ray Horton,"Canton, Ohio",Tom Benson Hall of Fame Stadium
4,South Division,South Division,South Division,South Division
5,Birmingham Stallions,Skip Holtz,"Birmingham, Alabama",Protective Stadium
6,New Orleans Breakers,John DeFilippo,"Birmingham, Alabama",Protective Stadium
7,Houston Gamblers,Curtis Johnson,"Memphis, Tennessee",Simmons Bank Liberty Stadium
8,Memphis Showboats,Todd Haley,"Memphis, Tennessee",Simmons Bank Liberty Stadium


In [40]:
df

,Unnamed: 0,Day,Date,VenueType,team2,score,result,OT,Location,Venue,Attendance,team1,season,score2,score1
107,110,Sun,1983-03-06,A,WashingtonFederals,28-7,W,NaN,"Washington, DC",RFK Stadium,38010.0,ChicagoBlitz,1983.0,7.0,28.0
105,194,Sun,1983-03-06,H,NewJerseyGenerals,20-15,W,NaN,"Los Angeles, CA",Memorial Coliseum,34002.0,LosAngelesExpress,1983.0,15.0,20.0
108,214,Sun,1983-03-06,H,PhiladelphiaStars,7-13,L,NaN,"Denver, CO",Mile High Stadium,45102.0,DenverGold,1983.0,13.0,7.0
109,132,Sun,1983-03-06,H,BostonBreakers,21-17,W,NaN,"Tampa, FL",Tampa Stadium,42437.0,TampaBayBandits,1983.0,17.0,21.0
106,234,Sun,1983-03-06,H,OaklandInvaders,0-24,L,NaN,"Tempe, AZ",Sun Devil Stadium,45167.0,ArizonaWranglers,1983.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,1107,Sat,2022-06-25,A,NewJerseyGenerals,19-14,W,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,PhiladelphiaStars,2022.0,14.0,19.0
418,1055,Sat,2022-06-25,A,BirminghamStallions,17-31,L,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,NewOrleansBreakers,2022.0,31.0,17.0
419,1093,Sat,2022-06-25,A,BirminghamStallions,17-31,L,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,NewOrleansBreakers,2022.0,31.0,17.0
472,1017,Sun,2022-07-03,H,BirminghamStallions,33-30,W,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,PhiladelphiaStars,2022.0,30.0,33.0


In [29]:
hfaz

0.0

In [23]:
np.mean(hfa)

0.0

In [32]:
eloLeague.ratingDict['NewOrleansBreakers']

1549.0124369982996

In [33]:
eloLeague.ratingDict['PittsburghMaulers']

1450.9875630017004

In [3]:
print(100/(w+l)*w)

42.857142857142854


In [105]:
df[(df.team1=='NewOrleansBreakers')]

,Day,Date,VenueType,team2,score,result,OT,Location,Venue,Attendance,team1,season,score2,score1
153,Mon,1984-04-02 00:00:00,A,BirminghamStallions,17-31,L,NaN,"Birmingham, AL",Legion Field,28100.0,NewOrleansBreakers,1984.0,0.0,0.0
217,Sun,1984-04-08 00:00:00,H,PittsburghMaulers,27-24,W,NaN,"New Orleans, LA",Louisiana Superdome,39315.0,NewOrleansBreakers,1984.0,0.0,0.0
139,Mon,1984-04-16 00:00:00,H,TampaBayBandits,13-35,L,NaN,"New Orleans, LA",Louisiana Superdome,35634.0,NewOrleansBreakers,1984.0,0.0,0.0
183,Fri,1984-04-27 00:00:00,A,PhiladelphiaStars,0-35,L,NaN,"Philadelphia, PA",Veterans Stadium,34011.0,NewOrleansBreakers,1984.0,0.0,0.0
272,Sun,1984-05-20 00:00:00,A,TampaBayBandits,20-31,L,NaN,"Tampa, FL",Tampa Stadium,42592.0,NewOrleansBreakers,1984.0,0.0,0.0
301,Sun,1984-05-27 00:00:00,H,BirminghamStallions,14-31,L,NaN,"New Orleans, LA",Louisiana Superdome,23748.0,NewOrleansBreakers,1984.0,0.0,0.0
348,Sun,1984-06-10 00:00:00,A,NewJerseyGenerals,21-31,L,NaN,"East Rutherford, NJ",Giants Stadium,23114.0,NewOrleansBreakers,1984.0,0.0,0.0
409,Sun,1984-06-24 00:00:00,A,WashingtonFederals,17-20,L,NaN,"Washington, DC",RFK Stadium,6386.0,NewOrleansBreakers,1984.0,0.0,0.0
178,Sun,2022-04-24 00:00:00,A,TampaBayBandits,34-3,W,NaN,"Birmingham, AL",Protective Stadium,NaN,NewOrleansBreakers,2022.0,0.0,0.0
203,Sat,2022-04-30 00:00:00,N,BirminghamStallions,13-22,L,NaN,"Birmingham, AL",Protective Stadium,NaN,NewOrleansBreakers,2022.0,0.0,0.0


In [103]:
l2s={k: v for k, v in sorted(eloLeague.ratingDict.items(), key=lambda item: item[1],reverse=True)}
for key in l2s.keys():
    print(key,eloLeague.ratingDict[key])

BirminghamStallions 1559.5008492600314
PhiladelphiaStars 1545.4675048991512
DenverGold 1505.0000106379102
HoustonGamblers 1505.0000104185985
PortlandBreakers 1505.0000075264434
BostonBreakers 1504.999999547811
WashingtonFederals 1504.999999547811
BaltimoreStars 1504.999999547811
NewOrleansBreakers 1504.999999547811
SanAntonioGunslingers 1504.999999547811
MemphisShowboats 1504.999999547811
JacksonvilleBulls 1504.999999547811
MichiganPanthers 1504.999999547811
PittsburghMaulers 1504.999999547811
ArizonaWranglers 1504.999999547811
OklahomaOutlaws 1504.999999547811
ChicagoBlitz 1504.999999547811
ArizonaOutlaws 1504.999991569179
OrlandoRenegades 1504.9999913132856
LosAngelesExpress 1504.9999886770238
OaklandInvaders 1504.9999884577119
TampaBayBandits 1464.5325024309964
NewJerseyGenerals 1450.4991498355907


In [106]:
df.dtypes

Day            object
Date           string
VenueType      object
team2          string
score          object
result         object
OT             object
Location       object
Venue          object
Attendance    float64
team1          string
season        float64
score2        float64
score1        float64
dtype: object